In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [2]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

In [4]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [5]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [6]:
print(dictionary[66])

faith


In [7]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.015*"weaver" + 0.013*"allocation" + 0.012*"bandwidth" + 0.012*"expose"')
(1, '0.019*"game" + 0.017*"team" + 0.012*"games" + 0.012*"year"')
(2, '0.011*"people" + 0.010*"would" + 0.006*"government" + 0.005*"right"')
(3, '0.014*"runs" + 0.011*"ball" + 0.010*"year" + 0.008*"pitching"')
(4, '0.045*"stream" + 0.009*"captain" + 0.008*"laptop" + 0.008*"astros"')
(5, '0.027*"period" + 0.013*"power" + 0.011*"scorer" + 0.007*"play"')
(6, '0.021*"file" + 0.012*"program" + 0.009*"window" + 0.009*"output"')
(7, '0.013*"would" + 0.011*"people" + 0.009*"think" + 0.007*"know"')
(8, '0.018*"pitt" + 0.018*"gordon" + 0.016*"banks" + 0.015*"soon"')
(9, '0.012*"colorado" + 0.008*"seagate" + 0.007*"shock" + 0.007*"japanese"')
(10, '0.010*"book" + 0.010*"article" + 0.007*"books" + 0.007*"posting"')
(11, '0.013*"like" + 0.009*"time" + 0.008*"good" + 0.008*"would"')
(12, '0.025*"food" + 0.012*"smokeless" + 0.012*"nist" + 0.010*"ncsl"')
(13, '0.016*"space" + 0.007*"information" + 0.006*"data" + 0.006*"res

In [8]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)

In [9]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(2, 0.7126882), (7, 0.18043402), (19, 0.0931525)]
1 번째 문서의 topic 비율은 [(3, 0.027568934), (7, 0.83078647), (14, 0.027561428), (18, 0.09300533)]
2 번째 문서의 topic 비율은 [(2, 0.71795857), (7, 0.26750332)]
3 번째 문서의 topic 비율은 [(7, 0.21305376), (8, 0.041185897), (9, 0.016395262), (10, 0.020110868), (11, 0.3044418), (18, 0.39384028)]
4 번째 문서의 topic 비율은 [(1, 0.3180421), (5, 0.28076062), (6, 0.04126207), (7, 0.33029193)]


c:\users\윤보경\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

c:\users\윤보경\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

c:\users\윤보경\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,2.0,0.7126,"[(2, 0.7126213), (7, 0.18049982), (19, 0.09315..."
1,1,7.0,0.8308,"[(3, 0.02756895), (7, 0.8307869), (14, 0.02756..."
2,2,2.0,0.7179,"[(2, 0.71791804), (7, 0.2675438)]"
3,3,18.0,0.3938,"[(7, 0.21306685), (8, 0.041185893), (9, 0.0163..."
4,4,7.0,0.3300,"[(1, 0.3181106), (5, 0.28074548), (6, 0.041472..."
5,5,7.0,0.5546,"[(7, 0.5546116), (10, 0.4062368)]"
6,6,11.0,0.2661,"[(3, 0.13018247), (4, 0.014756404), (6, 0.1774..."
7,7,7.0,0.5330,"[(2, 0.43391842), (7, 0.5330334), (14, 0.01908..."
8,8,7.0,0.6742,"[(7, 0.6741697), (15, 0.13241398), (16, 0.1683..."
9,9,11.0,0.6883,"[(2, 0.09146587), (9, 0.028890252), (11, 0.688..."
